In [1]:
!pip install azureml-core

     |████████████████████████████████| 2.5 MB 5.0 MB/s 
     |████████████████████████████████| 3.0 MB 75.3 MB/s 
     |████████████████████████████████| 55 kB 3.2 MB/s 
     |████████████████████████████████| 146 kB 74.5 MB/s 
     |████████████████████████████████| 59 kB 6.5 MB/s 
     |████████████████████████████████| 2.2 MB 58.4 MB/s 
     |████████████████████████████████| 85 kB 3.9 MB/s 
     |████████████████████████████████| 86 kB 5.5 MB/s 
     |████████████████████████████████| 40 kB 5.4 MB/s 
     |████████████████████████████████| 141 kB 51.1 MB/s 
     |████████████████████████████████| 78 kB 5.9 MB/s 
     |████████████████████████████████| 1.8 MB 66.4 MB/s 
     |████████████████████████████████| 412 kB 73.3 MB/s 
     |████████████████████████████████| 55 kB 3.2 MB/s 
     |████████████████████████████████| 94 kB 2.7 MB/s 
     |████████████████████████████████| 210 kB 60.7 MB/s 
     |████████████████████████████████| 928 kB 62.0 MB/s 
     |█████████████████████████

In [2]:
!pip install azureml-train-automl

     |████████████████████████████████| 136 kB 5.2 MB/s 
     |████████████████████████████████| 277 kB 54.6 MB/s 
     |████████████████████████████████| 2.1 MB 53.3 MB/s 
     |████████████████████████████████| 222 kB 47.9 MB/s 
     |████████████████████████████████| 7.4 MB 26.1 MB/s 
     |████████████████████████████████| 371 kB 76.2 MB/s 
     |████████████████████████████████| 7.1 MB 46.1 MB/s 
     |████████████████████████████████| 219 kB 60.1 MB/s 
     |████████████████████████████████| 696 kB 56.2 MB/s 
     |████████████████████████████████| 10.4 MB 43.9 MB/s 
     |████████████████████████████████| 20.1 MB 68.0 MB/s 
     |████████████████████████████████| 8.4 MB 31.9 MB/s 
     |████████████████████████████████| 43 kB 1.6 MB/s 
     |████████████████████████████████| 70 kB 7.7 MB/s 
     |████████████████████████████████| 4.5 MB 44.2 MB/s 
     |████████████████████████████████| 131 kB 65.2 MB/s 
     |████████████████████████████████| 114 kB 78.3 MB/s 
     |███████████

In [11]:
!pip install azureml-widgets

     |████████████████████████████████| 14.2 MB 5.0 MB/s 
     |████████████████████████████████| 356 kB 52.2 MB/s 


In [3]:
import azureml.core
print(azureml.core.VERSION)

1.37.0


In [12]:
import azureml.core

from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core import Dataset
from azureml.core.experiment import Experiment
from azureml.core.model import Model
from azureml.train.automl import AutoMLConfig

from datetime import datetime
from dateutil.relativedelta import relativedelta

import logging
import os 
import warnings
warnings.simplefilter("ignore")

**Create Workspace**

In [ ]:
ws = Workspace.create(
    name="Diabetes_Detection", 
    subscription_id=private_subscription_id,
    resource_group="future_ready",
    create_resource_group=True,
    location="eastus",
    sku='enterprise'
)

Performing interactive authentication. Please follow the instructions on the terminal.


To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code ENNFDSRXX to authenticate.


Interactive authentication successfully completed.
Deploying StorageAccount with name diabetesstorage4a9ddc64f.
Deploying KeyVault with name diabeteskeyvaultce3fa711.
Deploying AppInsights with name diabetesinsightsbef97f8e.
Deployed AppInsights with name diabetesinsightsbef97f8e. Took 75.06 seconds.
Deployed KeyVault with name diabeteskeyvaultce3fa711. Took 85.52 seconds.
Deployed StorageAccount with name diabetesstorage4a9ddc64f. Took 90.69 seconds.
Deploying Workspace with name Diabetes_Detection.
Deployed Workspace with name Diabetes_Detection. Took 82.9 seconds.


In [2]:
ws = Workspace.get(
    name="Diabetes_Detection", 
    subscription_id=private_subscription_id,
    resource_group="future_ready"
)

In [3]:
def_blob_store = ws.get_default_datastore()
def_blob_store.upload_files(["diabetes.csv"], target_path="data", overwrite=True)
diabetes_data = Dataset.Tabular.from_delimited_files(def_blob_store.path('./data/diabetes.csv'))
diabetes_data = diabetes_data.register(ws, 'diabetes_data', create_new_version=True)

"datastore.upload_files" is deprecated after version 1.0.69. Please use "FileDatasetFactory.upload_directory" instead. See Dataset API change notice at https://aka.ms/dataset-deprecation.


Uploading an estimated of 1 files
Uploading diabetes.csv
Uploaded diabetes.csv, 1 files out of an estimated total of 1
Uploaded 1 files


In [4]:
aml_compute_target = "diabetes-Cluster"
try:
  aml_compute = AmlCompute(ws, aml_compute_target)
  print("Existing Compute Traget Found")
except ComputeTargetException:
  print("Creating New Compute Instance... ")
  provisioning_config = AmlCompute.provisioning_configuration(vm_size = "STANDARD_D2_V2",
                                                              min_nodes = 0,
                                                              max_nodes = 4)
  ml_compute = ComputeTarget.create(ws, aml_compute_target, provisioning_config)
  ml_compute.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

print("Azure Machine Learning Compute Successfully Attached")

Existing Compute Traget Found
Azure Machine Learning Compute Successfully Attached


In [5]:
automl_settings = {
    "iterations": 20,
    "primary_metric": "AUC_weighted",
    "featurization": "auto",
    "verbosity": logging.INFO,
    "iteration_timeout_minutes": 10,
    "n_cross_validations": 2
}

In [6]:
automl_config = AutoMLConfig(task="classification",
                             debug_log="automated_ml_errors.log",
                             path="./content",
                             training_data=diabetes_data,
                             compute_target=aml_compute_target,
                             label_column_name="class",
                             **automl_settings)

In [7]:
experiment = Experiment(ws, "diabetes-experiment")
remote_run = experiment.submit(automl_config, show_output=True)
remote_run

Submitting remote run.
No run_configuration provided, running on diabetes-Cluster with default configuration
Running on remote compute: diabetes-Cluster


Experiment,Id,Type,Status,Details Page,Docs Page
diabetes-experiment,AutoML_08bd5b35-d2cb-4426-8313-a77c644d5977,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

********************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

**********************************************************************************

Experiment,Id,Type,Status,Details Page,Docs Page
diabetes-experiment,AutoML_08bd5b35-d2cb-4426-8313-a77c644d5977,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [9]:
best_run, model = remote_run.get_output()

In [ ]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

In [14]:
model_name = best_run.properties['model_name']
script_file_name = 'inference/score.py'
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'inference/score.py')

In [15]:
description = 'AutoML Diabetes model'
model = remote_run.register_model(model_name = model_name, description = description, tags = None)
print("Model ID", remote_run.model_id)

Model ID AutoML08bd5b35d18


In [16]:
from azureml.core.environment import Environment
from azureml.automl.core.shared import constants
best_run.download_file(constants.CONDA_ENV_FILE_PATH, 'myenv.yml')
myenv = Environment.from_conda_specification(name="myenv", file_path="myenv.yml")

In [17]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core.webservice import Webservice
from azureml.core.model import Model
from azureml.core.environment import Environment

inference_config = InferenceConfig(entry_script=script_file_name, environment=myenv)

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                               memory_gb = 1, 
                                               description = 'Diabetes Detection service')

aci_service_name = 'automl-sample-bankmarketing-all'
print(aci_service_name)
aci_service = Model.deploy(ws, aci_service_name, [model], inference_config, aciconfig)
aci_service.wait_for_deployment(True)
print(aci_service.state)

automl-sample-bankmarketing-all
Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-01-09 10:51:58+00:00 Creating Container Registry if not exists..
2022-01-09 11:01:58+00:00 Registering the environment..
2022-01-09 11:02:01+00:00 Building image..
2022-01-09 11:13:34+00:00 Generating deployment configuration.
2022-01-09 11:13:35+00:00 Submitting deployment to compute..
2022-01-09 11:13:40+00:00 Checking the status of deployment automl-sample-bankmarketing-all..
2022-01-09 11:17:54+00:00 Checking the status of inference endpoint automl-sample-bankmarketing-all.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


In [ ]:
import urllib.request
import json
import os
import ssl

def allowSelfSignedHttps(allowed):
    # bypass the server certificate verification on client side
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context

allowSelfSignedHttps(True) # this line is needed if you use self-signed certificate in your scoring service.

# Request data goes here
data = {
    "data": [
    {
      "Pregnancies": 0,
      "Glucose": 0,
      "BloodPressure": 0,
      "SkinThickness": 0,
      "Insulin": 0,
      "BMI": 0,
      "DiabetesPedigreeFunction": 0,
      "Age": 0
    }
  ],
  "method": "predict"
}

body = str.encode(json.dumps(data))

url = 'http://903ad508-e251-4475-8ae8-155a68dc106b.eastus.azurecontainer.io/score'
api_key = '' # Replace this with the API key for the web service
headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key)}

req = urllib.request.Request(url, body, headers)

try:
    response = urllib.request.urlopen(req)

    result = response.read()
    print(result)
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))

    # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
    print(error.info())